In [1]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc
from abenc_tg22 import DIPE_ABE, Inner_Product_TG22
from abenc_ph_mj18 import PH_ABE, mat_math, Inner_Product
from nizk import NIZK
from attack import RogueKeyAtt
from collections import defaultdict 
import time
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import numpy as np
import pickle


/home/guy1m0/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
group_tg = PairingGroup('SS512')
group_mj = PairingGroup('MNT159')
datasets = {}


## Bench Function

In [4]:
def benchmark(assump_size, n, datasets):

    # Pre-processing
    tg = {}
    mj = {}

    mj['total'] = 0
    tg['total'] = 0
    dipe = DIPE_ABE(n, group_tg)
    ph_abe = PH_ABE(n, assump_size, group_mj, mat_math())

    nizk = NIZK(group_mj)
    att = RogueKeyAtt(n, assump_size)
    print ('---------------------------------------------------------')
    print ('Setting with n:', n, "k:",assump_size)

    # sys setup
    start_time = time.time()
    pp_mj, _ = ph_abe.setup()
    elapsed_time = time.time() - start_time
    mj['sys'] = elapsed_time
    mj['total'] += elapsed_time

    start_time = time.time()
    pp_tg = dipe.setup()
    elapsed_time = time.time() - start_time
    tg['sys'] = elapsed_time
    tg['total'] += elapsed_time


    # Auth setup
    start_time = time.time()
    pks_mj, sks_mj = ph_abe.auth_setup(pp_mj)
    elapsed_time = time.time() - start_time
    mj['auth'] = elapsed_time
    mj['total'] += elapsed_time

    start_time = time.time()
    pks_tg, sks_tg = dipe.auth_setup(pp_tg)
    elapsed_time = time.time() - start_time
    tg['auth'] = elapsed_time
    tg['total'] += elapsed_time

    # Key Gen
    GID_tg = group_tg.random(ZR)
    GID_mj = group_mj.random(ZR)

    vec_x, vec_y = Inner_Product_TG22(group_tg).gen_x_y(n)
    start_time = time.time()
    D0, D1, K_tg = dipe.keygen(pp_tg, sks_tg, GID_tg, vec_x)
    elapsed_time = time.time() - start_time
    tg['keygen'] = elapsed_time
    tg['total'] += elapsed_time

    vec_x, vec_v = Inner_Product(group_mj).gen_x_v(n, assump_size)

    start_time = time.time()
    K_mj, _ = ph_abe.keygen(pp_mj, pks_mj, sks_mj, GID_mj, vec_v)
    elapsed_time = time.time() - start_time
    mj['keygen'] = elapsed_time
    mj['total'] += elapsed_time

    # Worst Case verify/prove
    start_time = time.time()
    for i in range(n):
        s_pairs, pis = nizk.prove_pk(pp_mj, pks_mj[str(i+1)], sks_mj[str(i+1)])
    elapsed_time = time.time() - start_time
    mj['prove'] = elapsed_time

    start_time = time.time()
    for i in range(n):
        check = nizk.verify_pk(pp_mj, s_pairs, pis)
        if not check:
            print ("Check not pass for AA:", str(i+1))
            break
    elapsed_time = time.time() - start_time
    mj['verify'] = elapsed_time

    # AD's setup
    start_time = time.time()
    ad = vec_v.index(0) + 1

    ad_vec_v = [0] * (n-1) + [1]
    pks_mj = att.pks_update(ad, pks_mj)
    elapsed_time = time.time() - start_time
    mj['ad_setup'] = elapsed_time
    #print ("Adv's setup: ", elapsed_time)

    # AD KeyGen
    GID_ad = group_mj.random(ZR)
    K_,ad_key_gen = ph_abe.keygen(pp_mj, pks_mj, sks_mj, GID_ad, ad_vec_v, ad=ad)
    mj['ad_keygen'] = ad_key_gen

    # Encryption
    M_tg = group_tg.random(GT)
    start_time = time.time()
    C_tg = dipe.encrypt(pp_tg, pks_tg, vec_y, M_tg)
    elapsed_time = time.time() - start_time
    tg['encrypt'] = elapsed_time
    tg['total'] += elapsed_time

    M_mj = group_mj.random(GT)
    start_time = time.time()    
    #print ('M:', M)
    C_mj, vec_s = ph_abe.encrypt(pp_mj, pks_mj, vec_x, M_mj)
    elapsed_time = time.time() - start_time
    mj['encrypt'] = elapsed_time
    mj['total'] += elapsed_time

    # Decryption
    start_time = time.time()
    M_ = dipe.decrypt(D0, D1, K_tg, C_tg, vec_y)
    if M_ != M_tg:
        print ('Error in decrypt M (usr): ', M_)
    elapsed_time = time.time() - start_time
    #print ("DU's cost: ", elapsed_time)
    tg['decrypt'] = elapsed_time
    tg['total'] += elapsed_time


    start_time = time.time()
    M_ = ph_abe.decrypt(K_mj, C_mj, vec_v, pp_mj)
    if M_ != M_mj:
        print ('Error in decrypt M (usr): ', M_)
    elapsed_time = time.time() - start_time
    #print ("DU's cost: ", elapsed_time)
    mj['decrypt'] = elapsed_time
    mj['total'] += elapsed_time

    # AD decrypt
    start_time = time.time()
    tmp = ph_abe.decrypt(K_, C_mj, ad_vec_v, pp_mj)
    omega = att.gen_omega(K_,C_mj)
    M_ =  tmp * mat_math().prod(omega)
    if M_ != M_mj:
        print ('Error in decrypt M (adv): ', M_)
    elapsed_time = time.time() - start_time
    mj['ad_cancel_out'] = elapsed_time
    #print ("Ad's cost: ", data['ad_setup'] + data['ad_keygen'] + data['ad_cancel_out'])

    print ("MJ's sys cost: ", mj['total'])
    print ("TG's sys cost: ", tg['total'])
    print ("Enhd sys cost: ", mj['total'] + mj['prove'] + mj['verify'])

    datasets[f'k_{str(assump_size)}_n_{str(n)}']= {'mj18':mj, 'tg22':tg}

    return datasets
    # datasets[str(k)]['n']=data
    # datasets[str(k)]['seq'].append(data)




In [21]:
def benchmark_parms(assump_size, ns, n_DO, n_M, datasets):
  
    nizk = NIZK(group_mj)

    for n in ns:
        mj = {}
        mj['pre_use'] = 0
        ph_abe = PH_ABE(n, assump_size, group_mj, mat_math())

        att = RogueKeyAtt(n, assump_size)
        print ('---------------------------------------------------------')
        print ('Setting with n:', n, "k:",assump_size)

        # sys setup
        start_time = time.time()
        pp_mj, _ = ph_abe.setup()
        elapsed_time = time.time() - start_time
        mj['sys'] = elapsed_time
        mj['pre_use'] += elapsed_time

        # Auth setup
        start_time = time.time()
        pks_mj, sks_mj = ph_abe.auth_setup(pp_mj)
        elapsed_time = time.time() - start_time
        mj['auth'] = elapsed_time
        mj['pre_use'] += elapsed_time

        GID_mj = group_mj.random(ZR)
        GID_ad = group_mj.random(ZR)
        vec_x, vec_v = Inner_Product(group_mj).gen_x_v(n, assump_size)


        # Worst Case verify/prove
        start_time = time.time()
        for i in range(n):
            s_pairs, pis = nizk.prove_pk(pp_mj, pks_mj[str(i+1)], sks_mj[str(i+1)])
        elapsed_time = time.time() - start_time
        mj['prove'] = elapsed_time

        start_time = time.time()
        for i in range(n):
            check = nizk.verify_pk(pp_mj, s_pairs, pis)
            if not check:
                print ("Check not pass for AA:", str(i+1))
                break
        elapsed_time = time.time() - start_time
        mj['verify'] = elapsed_time

        # AD's setup
        start_time = time.time()
        ad = vec_v.index(0) + 1

        ad_vec_v = [0] * (n-1) + [1]
        pks_mj = att.pks_update(ad, pks_mj)
        elapsed_time = time.time() - start_time
        mj['ad_setup'] = elapsed_time


        mj['keygen'] = []
        mj['ad_keygen'] = []

        # Performance result of varing number of Data Owner
        for _ in range(n_DO):
            # KeyGen
            start_time = time.time()
            K_mj, _ = ph_abe.keygen(pp_mj, pks_mj, sks_mj, GID_mj, vec_v)
            elapsed_time = time.time() - start_time
            mj['keygen'].append(elapsed_time)


            K_,ad_key_gen = ph_abe.keygen(pp_mj, pks_mj, sks_mj, GID_ad, ad_vec_v, ad=ad)
            mj['ad_keygen'].append(ad_key_gen)

        mj['encrypt'] = []
        mj['decrypt'] = []
        mj['ad_cancel_out'] = []

        # Performance result of varing number of Message
        for _ in range(n_M):
            # Encryption
            M_mj = group_mj.random(GT)
            start_time = time.time()    

            C_mj, vec_s = ph_abe.encrypt(pp_mj, pks_mj, vec_x, M_mj)
            elapsed_time = time.time() - start_time
            mj['encrypt'].append(elapsed_time)

            # Decryption
            start_time = time.time()
            M_ = ph_abe.decrypt(K_mj, C_mj, vec_v, pp_mj)
            if M_ != M_mj:
                print ('Error in decrypt M (usr): ', M_)
            elapsed_time = time.time() - start_time
            #print ("DU's cost: ", elapsed_time)
            mj['decrypt'].append(elapsed_time)

            # AD decrypt
            start_time = time.time()
            tmp = ph_abe.decrypt(K_, C_mj, ad_vec_v, pp_mj)
            omega = att.gen_omega(K_,C_mj)
            M_ =  tmp * mat_math().prod(omega)
            if M_ != M_mj:
                print ('Error in decrypt M (adv): ', M_)
            elapsed_time = time.time() - start_time
            mj['ad_cancel_out'].append(elapsed_time)

        print ("MJ pre_use cost:", mj['pre_use'])
        print ("Enhd extra cost:", mj['prove']+mj['verify'])
        print ("Total KeyGen cost: ", sum(mj['keygen']))
        print ("Total Enc/Dec cost:", sum(mj['decrypt'])+sum(mj['encrypt']))
        print ("Total Attack cost: ", mj['ad_setup']+sum(mj['ad_keygen'])+sum(mj['ad_cancel_out']))

        datasets[f'k_{str(assump_size)}_n_{str(n)}']= mj

    return datasets


## Data Collection

### Varing the number of AA

In [ ]:
for n in range(5,126,3):
    datasets = benchmark(2, n, datasets)

---------------------------------------------------------
Setting with n: 5 k: 2
MJ's sys cost:  0.16038298606872559
TG's sys cost:  0.04531383514404297
Enhd sys cost:  0.2661724090576172
---------------------------------------------------------
Setting with n: 8 k: 2
MJ's sys cost:  0.2791147232055664
TG's sys cost:  0.11244702339172363
Enhd sys cost:  0.44754457473754883
---------------------------------------------------------
Setting with n: 11 k: 2
MJ's sys cost:  0.4374237060546875
TG's sys cost:  0.21063613891601562
Enhd sys cost:  0.6670856475830078
---------------------------------------------------------
Setting with n: 14 k: 2
MJ's sys cost:  0.8126533031463623
TG's sys cost:  0.4559481143951416
Enhd sys cost:  1.2177114486694336
---------------------------------------------------------
Setting with n: 17 k: 2
MJ's sys cost:  1.1627342700958252
TG's sys cost:  0.7219388484954834
Enhd sys cost:  1.6501593589782715
---------------------------------------------------------
Sett

In [ ]:
with open('data_136K_mj18_tg22_enhd_k_2_n_5-125.pkl','wb') as file:
    pickle.dump(datasets, file)

### Varing the number of DO and M

In [27]:
datasets_parms = {}
datasets_parms = benchmark_parms(2, [5,35,65,95,125],10,100,datasets_parms)

---------------------------------------------------------
Setting with n: 5 k: 2
MJ pre_use cost: 0.07399106025695801
Enhd extra cost: 0.14234519004821777
Total KeyGen cost:  0.9849295616149902
Total Enc/Dec cost: 3.9162051677703857
Total Attack cost:  1.927544116973877
---------------------------------------------------------
Setting with n: 35 k: 2
MJ pre_use cost: 0.4483616352081299
Enhd extra cost: 1.0203094482421875
Total KeyGen cost:  34.14306879043579
Total Enc/Dec cost: 20.066277027130127
Total Attack cost:  2.1044013500213623
---------------------------------------------------------
Setting with n: 65 k: 2
MJ pre_use cost: 0.83005690574646
Enhd extra cost: 1.8759098052978516
Total KeyGen cost:  115.62151074409485
Total Enc/Dec cost: 36.39972996711731
Total Attack cost:  2.2559685707092285
---------------------------------------------------------
Setting with n: 95 k: 2
MJ pre_use cost: 1.2162103652954102
Enhd extra cost: 2.486508846282959
Total KeyGen cost:  197.67919754981995

In [31]:
with open('data_136K_mj18_k_2_n_5_35_65_95_DO_5_M_100.pkl','wb') as file:
    pickle.dump(datasets_parms, file)

In [32]:
datasets_parms

{'k_2_n_5': {'pre_use': 0.07399106025695801,
  'sys': 0.007803201675415039,
  'auth': 0.06618785858154297,
  'prove': 0.04827737808227539,
  'verify': 0.09406781196594238,
  'ad_setup': 6.413459777832031e-05,
  'keygen': [0.09904909133911133,
   0.09797477722167969,
   0.09867620468139648,
   0.09831786155700684,
   0.09833431243896484,
   0.09847664833068848,
   0.09848809242248535,
   0.09871649742126465,
   0.0985414981842041,
   0.09835457801818848],
  'ad_keygen': [0.0077667236328125,
   0.007720232009887695,
   0.007679462432861328,
   0.007659435272216797,
   0.00765681266784668,
   0.0077245235443115234,
   0.00777125358581543,
   0.007688045501708984,
   0.007686138153076172,
   0.00767064094543457],
  'encrypt': [0.026940107345581055,
   0.026610612869262695,
   0.026439189910888672,
   0.02639460563659668,
   0.02702188491821289,
   0.026550769805908203,
   0.026076078414916992,
   0.02687859535217285,
   0.026948213577270508,
   0.02683711051940918,
   0.026742935180664062,